# COURSE: A deep understanding of deep learning
## SECTION: Overfitting, cross-validation, regularization
### LECTURE: Cross-validation -- manual separation
#### TEACHER: Mike X Cohen, sincxpress.com
##### COURSE URL: udemy.com/course/deeplearning_x/?couponCode=202401

In [30]:
# import libraries
import torch
import torch.nn as nn
import numpy as np

In [31]:
# import dataset
import pandas as pd
iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')


# convert from pandas dataframe to tensor
data = torch.tensor( iris[iris.columns[0:4]].values ).float()

# transform species to number
labels = torch.zeros(len(data), dtype=torch.long)
# labels[iris.species=='setosa'] = 0 # don't need!
labels[iris.species=='versicolor'] = 1
labels[iris.species=='virginica'] = 2

In [86]:
nLabels = len(np.unique(labels.numpy()))

# Separate data into train and test

In [32]:
#  (no devset here)

# how many training examples
propTraining = .8 # in proportion, not percent
nTraining = int(len(labels)*propTraining)

# initialize a boolean vector to select data and labels
traintestBool = np.zeros(len(labels),dtype=bool)

# is this the correct way to select samples?
#traintestBool[range(nTraining)] = True

# this is better, but why?
#items2use4train = np.random.choice(range(len(labels)),nTraining,replace=False)
#traintestBool[items2use4train] = True

traintestBool

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False, False, False, False, False, False,
       False, False,

In [129]:
full_train = []

for i in range(nLabels):
    train_group = train_numbers -1 + 50 * i
    full_train.extend(train_group)  # Use extend to add elements to the list

print(full_train)

[np.int64(4), np.int64(1), np.int64(15), np.int64(20), np.int64(45), np.int64(31), np.int64(49), np.int64(36), np.int64(32), np.int64(3), np.int64(29), np.int64(21), np.int64(0), np.int64(30), np.int64(34), np.int64(22), np.int64(39), np.int64(12), np.int64(46), np.int64(48), np.int64(35), np.int64(47), np.int64(26), np.int64(7), np.int64(19), np.int64(11), np.int64(18), np.int64(43), np.int64(40), np.int64(33), np.int64(37), np.int64(17), np.int64(28), np.int64(44), np.int64(25), np.int64(27), np.int64(2), np.int64(10), np.int64(8), np.int64(5), np.int64(54), np.int64(51), np.int64(65), np.int64(70), np.int64(95), np.int64(81), np.int64(99), np.int64(86), np.int64(82), np.int64(53), np.int64(79), np.int64(71), np.int64(50), np.int64(80), np.int64(84), np.int64(72), np.int64(89), np.int64(62), np.int64(96), np.int64(98), np.int64(85), np.int64(97), np.int64(76), np.int64(57), np.int64(69), np.int64(61), np.int64(68), np.int64(93), np.int64(90), np.int64(83), np.int64(87), np.int64(67),

In [130]:
traintestBool2 = np.zeros(len(labels), dtype=bool)
traintestBool2[full_train] = True

In [131]:
# test whether it's balanced
print('Average of full data:')
print( torch.mean(labels.float()) ) # =1 by definition
print(' ')

print('Average of training data:')
print( torch.mean(labels[traintestBool2].float()) ) # should be 1...
print(' ')

print('Average of test data:')
print( torch.mean(labels[~traintestBool2].float()) ) # should also be 1...

Average of full data:
tensor(1.)
 
Average of training data:
tensor(1.)
 
Average of test data:
tensor(1.)


In [109]:
# test whether it's balanced
print('Average of full data:')
print( torch.mean(labels.float()) ) # =1 by definition
print(' ')

print('Average of training data:')
print( torch.mean(labels[traintestBool].float()) ) # should be 1...
print(' ')

print('Average of test data:')
print( torch.mean(labels[~traintestBool].float()) ) # should also be 1...

Average of full data:
tensor(1.)
 
Average of training data:
tensor(1.)
 
Average of test data:
tensor(1.)


In [35]:
# create the ANN model

# model architecture
ANNiris = nn.Sequential(
    nn.Linear(4,64),   # input layer
    nn.ReLU(),         # activation unit
    nn.Linear(64,64),  # hidden layer
    nn.ReLU(),         # activation unit
    nn.Linear(64,3),   # output units
      )

# loss function
lossfun = nn.CrossEntropyLoss()

# optimizer
optimizer = torch.optim.SGD(ANNiris.parameters(),lr=.01)

In [36]:
ANNiris

Sequential(
  (0): Linear(in_features=4, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=3, bias=True)
)

In [37]:
# entire dataset
print( data.shape )

# training set
print( data[traintestBool,:].shape )

# test set
print( data[~traintestBool,:].shape )

torch.Size([150, 4])
torch.Size([120, 4])
torch.Size([30, 4])


# Train and test the model

In [38]:
# train the model

numepochs = 1000

# initialize losses
losses = torch.zeros(numepochs)
ongoingAcc = []

# loop over epochs
for epochi in range(numepochs):

  # forward pass
  yHat = ANNiris(data[traintestBool,:])

  # compute accuracy (note: denser than previous code!)
  ongoingAcc.append( 100*torch.mean(
              (torch.argmax(yHat,axis=1) == labels[traintestBool]).float()) )

  # compute loss
  loss = lossfun(yHat,labels[traintestBool])
  losses[epochi] = loss

  # backprop
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

In [39]:
# compute train and test accuracies

# final forward pass USING TRAINING DATA
predictions = ANNiris(data[traintestBool,:])
trainacc = 100*torch.mean((torch.argmax(predictions,axis=1) == labels[traintestBool]).float())


# final forward pass USING TEST DATA!
predictions = ANNiris(data[~traintestBool,:])
testacc = 100*torch.mean((torch.argmax(predictions,axis=1) == labels[~traintestBool]).float())

In [40]:
# report accuracies

print('Final TRAIN accuracy: %g%%' %trainacc)
print('Final TEST accuracy:  %g%%' %testacc)

Final TRAIN accuracy: 98.3333%
Final TEST accuracy:  73.3333%


In [41]:
# normally also inspect losses and accuracy by epoch, etc etc etc.

In [46]:
labels[~traintestBool]

tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2])

In [44]:
torch.argmax(predictions,axis=1)

tensor([2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2,
        2, 2, 2, 2, 2, 2])

In [48]:
# final forward pass USING TEST DATA!
predictions_full = ANNiris(data)
testacc = 100*torch.mean((torch.argmax(predictions_full,axis=1) == labels).float())

In [49]:
testacc

tensor(93.3333)

In [51]:
# Convert labels and predictions_full to numpy arrays
labels_np = labels.numpy()
predictions_full_np = torch.argmax(predictions_full, axis=1).numpy()

# Create a dataframe
df_results = pd.DataFrame({
    'Label': labels_np,
    'Prediction': predictions_full_np
})

df_results

,Label,Prediction
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
145,2,2
146,2,2
147,2,2
148,2,2


In [58]:
df_results["Correct"] = (df_results["Label"] == df_results["Prediction"])

In [60]:
df_results["Correct"] = df_results["Correct"].astype(int)

In [62]:
df_results.groupby("Label").mean()

,Prediction,Correct
Label,,
0,0.00,1.00
1,1.02,0.98
2,1.82,0.82


# Additional explorations

In [108]:
def balanced_train_test_split(data, labels, propTraining):
    unique_labels = torch.unique(labels)
    train_indices = []
    test_indices = []

    for label in unique_labels:
        label_indices = torch.where(labels == label)[0]
        n_train = int(len(label_indices) * propTraining)
        train_indices.extend(np.random.choice(label_indices, n_train, replace=False))
        test_indices.extend(list(set(label_indices.numpy()) - set(train_indices)))

    return train_indices, test_indices

train_indices, test_indices = balanced_train_test_split(data, labels, propTraining)

# Create boolean mask for train/test split
traintestBool = np.zeros(len(labels), dtype=bool)
traintestBool[train_indices] = True

In [123]:
train_indices

[np.int64(40),
 np.int64(16),
 np.int64(7),
 np.int64(41),
 np.int64(25),
 np.int64(44),
 np.int64(46),
 np.int64(1),
 np.int64(28),
 np.int64(20),
 np.int64(13),
 np.int64(14),
 np.int64(21),
 np.int64(26),
 np.int64(35),
 np.int64(11),
 np.int64(10),
 np.int64(34),
 np.int64(17),
 np.int64(27),
 np.int64(47),
 np.int64(18),
 np.int64(23),
 np.int64(38),
 np.int64(37),
 np.int64(5),
 np.int64(31),
 np.int64(32),
 np.int64(42),
 np.int64(33),
 np.int64(6),
 np.int64(12),
 np.int64(48),
 np.int64(19),
 np.int64(36),
 np.int64(49),
 np.int64(8),
 np.int64(30),
 np.int64(4),
 np.int64(2),
 np.int64(50),
 np.int64(81),
 np.int64(82),
 np.int64(80),
 np.int64(61),
 np.int64(77),
 np.int64(93),
 np.int64(70),
 np.int64(83),
 np.int64(59),
 np.int64(60),
 np.int64(88),
 np.int64(91),
 np.int64(84),
 np.int64(58),
 np.int64(96),
 np.int64(62),
 np.int64(95),
 np.int64(90),
 np.int64(99),
 np.int64(51),
 np.int64(78),
 np.int64(66),
 np.int64(67),
 np.int64(75),
 np.int64(55),
 np.int64(87),
 n

In [116]:
torch.where(labels == 0)[0]

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

In [117]:
torch.where(labels == 0)

(tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
         36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),)

In [119]:

unique_labels = torch.unique(labels)
train_indices = []
test_indices = []

for label in unique_labels:
    print(label)
    label_indices = torch.where(labels == label)[0]
    print(label_indices)
    n_train = int(len(label_indices) * propTraining)
    print(n_train)
    train_indices.extend(np.random.choice(label_indices, n_train, replace=False))
    print(train_indices)
    test_indices.extend(list(set(label_indices.numpy()) - set(train_indices)))
    print(test_indices)



tensor(0)
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])
40
[np.int64(40), np.int64(16), np.int64(7), np.int64(41), np.int64(25), np.int64(44), np.int64(46), np.int64(1), np.int64(28), np.int64(20), np.int64(13), np.int64(14), np.int64(21), np.int64(26), np.int64(35), np.int64(11), np.int64(10), np.int64(34), np.int64(17), np.int64(27), np.int64(47), np.int64(18), np.int64(23), np.int64(38), np.int64(37), np.int64(5), np.int64(31), np.int64(32), np.int64(42), np.int64(33), np.int64(6), np.int64(12), np.int64(48), np.int64(19), np.int64(36), np.int64(49), np.int64(8), np.int64(30), np.int64(4), np.int64(2)]
[np.int64(0), np.int64(3), np.int64(39), np.int64(9), np.int64(43), np.int64(45), np.int64(15), np.int64(22), np.int64(24), np.int64(29)]
tensor(1)
tensor([50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 

In [42]:
# 1) Randomly assigning data samples to be in the train vs test phase produced a statistical balance, but it was
#    not perfect. Write an algorithm that will guarantee a balance of flower types while also randomly assigning
#    samples to be in train vs. test.
#
# 2) Revert the code to its original form -- with the strong imbalance in flower types. Then train the model. What are
#    the train and test accuracies? Compute the accuracy separately for each type of flower to see whether the model
#    learned some categories, or whether it performed equally on all three categories. Are you surprised at the results?
#